In [14]:
from transformer_lens import HookedTransformer, HookedTransformerConfig
from transformer_lens.train import train, HookedTransformerTrainConfig
from datasets import load_dataset, Dataset, DatasetDict
import numpy as np
import torch as t
import os


In [15]:

device = t.device("cuda" if t.cuda.is_available() else "cpu")

n_layers = 6
d_model = 128

model_cfg = HookedTransformerConfig(
    n_layers = n_layers,
    d_model = d_model,
    d_head = 64,
    n_heads = 8,
    d_mlp = 128*4,
    d_vocab = 61,
    n_ctx = 59,
    act_fn="gelu",
    normalization_type="LN",
    device=device,
)

model = HookedTransformer(model_cfg).to(device)

train_cfg = HookedTransformerTrainConfig(
    lr=1e-4,
    batch_size=512,
    num_epochs=1,
    device=device,
    wandb=True,
    wandb_project_name="OthelloGPTTraining",
)


Moving model to device:  cuda


In [19]:
tokenized_data = t.tensor(np.load("data/board_seqs_int_small.npy"), dtype=t.long)
tokenized_data = tokenized_data[:, :59] # remove XX at the end
data_dict = {"tokens": tokenized_data.tolist()}

dataset = Dataset.from_dict(data_dict)
dataset.set_format(type="torch", columns=["tokens"])

train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})


In [17]:
# dataset = load_dataset("taufeeque/othellogpt", split="train")    


# dataset.set_format(type="torch", columns=["tokens"])

# # put the data on the device
# dataset = dataset.map(lambda x: {k: v.to(device) for k, v in x.items()})

# def truncate_tokens(example):
#     example['tokens'] = example['tokens'][:-1]
#     return example

# dataset = dataset.map(truncate_tokens)

In [20]:
train(model, train_cfg, train_dataset)
t.save(model.state_dict(), f"othello_gpt_{n_layers}_{d_model}.pt")

Moving model to device:  cuda


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1 Samples 512 Step 0 Loss 4.427001476287842


18it [00:05,  3.29it/s]
  0%|          | 0/1 [00:05<?, ?it/s]


KeyboardInterrupt: 